In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
DATA_FOLDER = '../data/'
DATA_ZIP = DATA_FOLDER + 'datasets.zip'

DATA_TRAIN_PATH = DATA_FOLDER + 'train.csv'
DATA_TRAIN_PATH_CLEAN = DATA_FOLDER + 'train_clean.csv'
DATA_TEST_PATH = DATA_FOLDER + 'test.csv' 

## Load the training data into feature matrix, class labels, and event ids:

In [23]:
from proj1_helpers import *
from split_data import *
from data_processing import *

y, tX, ids = load_csv_data(DATA_TRAIN_PATH)
y_clean, tX_clean, ids_clean = clean_training(y, tX, ids)


split_ratio = 0.8
tX_train, tX_validation, y_train, y_validation = split_data(tX, y, split_ratio)
tX_train_clean, tX_validation_clean, y_train_clean, y_validation_clean = split_data(tX_clean, y_clean, split_ratio)

In [17]:
print("Raw set : ")
row_format = "{:<15}" * 3
print(row_format.format("", "Training", "Validation"))
print(row_format.format("Features", str(tX_train.shape), str(tX_validation.shape)))
print(row_format.format("Labels", str(y_train.shape), str(y_validation.shape)))

print("\nClean set : ")
row_format = "{:<15}" * 3
print(row_format.format("", "Training", "Validation"))
print(row_format.format("Features", str(tX_train_clean.shape), str(tX_validation_clean.shape)))
print(row_format.format("Labels", str(y_train_clean.shape), str(y_validation_clean.shape)))

Raw set : 
               Training       Validation     
Features       (200000, 30)   (50000, 30)    
Labels         (200000,)      (50000,)       

Clean set : 
               Training       Validation     
Features       (200000, 30)   (50000, 30)    
Labels         (200000,)      (50000,)       


## Testing every basic model on cleaned data

In [18]:
from pipeline import *

### (Stochastic) Gradient Descent

In [19]:
max_iters = 100
gamma = 0.01
initial_w = np.zeros(tX_train_clean.shape[1])

loss_gd, weights_gd = model_data(y_train_clean, tX_train_clean, 'gradient_descent', 
                              initial_w=initial_w, max_iters=max_iters, gamma=gamma)
loss_sgd, weights_sgd = model_data(y_train_clean, tX_train_clean, 'stochastic_gradient_descent',
                                                initial_w=initial_w, max_iters=max_iters, gamma=gamma)
# Not compared with the not cleaned set, because it needs normalizing to work
print("GD Training loss : {}".format(loss_gd))
print("GD Validation loss : {}".format(get_loss(y_validation_clean, tX_validation_clean, weights_gd)))
print("SGD Training loss : {}".format(loss_sgd))
print("SGD Validation loss : {}".format(get_loss(y_validation_clean, tX_validation_clean, weights_sgd)))

GD Training loss : 0.36888669205551355
GD Validation loss : 0.3702442421673406
SGD Training loss : 0.39074770033413814
SGD Validation loss : 0.37158028696807777


### Least Squares

In [20]:
weights, t_loss = model_data(y_train, tX_train, 'least_squares')
weights_clean, t_loss_clean = model_data(y_train_clean, tX_train_clean, 'least_squares')

print("Training loss : {}".format(t_loss))
print("Validation loss : {}".format(get_loss(y_validation, tX_validation, weights)))
print("Clean Training loss : {}".format(t_loss_clean))
print("Clean Validation loss : {}".format(get_loss(y_validation_clean, tX_validation_clean, weights_clean)))

Training loss : 0.3394268495585387
Validation loss : 0.3407678460856664
Clean Training loss : 0.34717878619915993
Clean Validation loss : 0.3487533341444347


### Ridge Regression

In [21]:
lambda_ = 1

weights, t_loss = model_data(y_train, tX_train, 'ridge_regression')
weights_clean, t_loss_clean = model_data(y_train_clean, tX_train_clean, 'ridge_regression')

print("Training loss : {}".format(t_loss))
print("Validation loss : {}".format(get_loss(y_validation, tX_validation, weights)))
print("Clean Training loss : {}".format(t_loss_clean))
print("Clean Validation loss : {}".format(get_loss(y_validation_clean, tX_validation_clean, weights_clean)))

Training loss : 0.3394307923454777
Validation loss : 0.340752914694699
Clean Training loss : 0.3471823515748675
Clean Validation loss : 0.3487417660590721


### (Penalized) Logistic Regression

In [22]:
max_iters = 500
gamma = 0.05
lambda_ = 0.1

weights_log, loss_log = model_data(y_train_clean, tX_train_clean, 'logistic_regression', 
                                   max_iters=max_iters, gamma=gamma) 
weights_p_log, loss_p_log = model_data(y_train_clean, tX_train_clean, 'regularized_logistic_regression', 
                                   max_iters=max_iters, gamma=gamma, lambda_=lambda_) 


print("Training loss : {}".format(loss_log))
print("Validation loss : {}".format(get_log_likelihood(y_validation_clean, tX_validation_clean, weights_log)))
print("Penalized Training loss : {}".format(loss_p_log))
print("Penalized Validation loss : {}".format(get_log_likelihood(y_validation_clean, tX_validation_clean, weights_p_log)))

Training loss : 0.4835365113696089
Validation loss : 0.5210679421032498
Penalized Training loss : 0.5819816450250063
Penalized Validation loss : 0.5432927690943854


## Generate predictions and save ouput in csv format for submission:

In [112]:
#This enable to run all cells without running this one when unnecessary
if False :
    y_test, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
    y_test_clean, tX_test_clean, ids_test_clean = clean_test(y_test, tX_test, ids_test)
    OUTPUT_PATH = DATA_FOLDER + 'submission.csv' 
    if True: ## Only for logistic regression, will de something nice later
        y_pred = predict_labels_logistic(weights_clean, tX_test_clean_poly)
    else:
        y_pred = predict_labels(weights_clean, tX_test_clean)
    print(y_pred)
    create_csv_submission(ids_test_clean, y_pred, OUTPUT_PATH)
else :
    print("Change False to True to generate prediction")

[-1. -1.  1. ...  1.  1.  1.]
